###  Use the gradient descent method to train a ReLU a shallow neural network for L^2 fitting tasks

Consider 1D  $L^2$-fitting problem on $\Omega = [-\pi,\pi]$: 
		\begin{equation}
			\min_{f_n \in \Sigma_n} \int_{-\pi}^\pi \frac{1}{2} | f(x) - f_n(x) |^2dx.
		\end{equation}
        
 $\Sigma_n$  is the spcae of  ReLU shallow neural network with $n$ neurons 
	\begin{equation*}
		\Sigma_n = \left\{ v(x) = \sum_{i=1}^n a_i \sigma(x + b_i) : a_i \in \mathbb{R},  ~~ b_i \in \mathbb{R}\right\}, \quad \sigma(x) = \max(0,x).
	\end{equation*}
    
The resulting nonlinear, nonconvex optimization problem
			\begin{equation} \label{snn-optimization}
			 \min_{a_i, b_i} \int_{-\pi}^{\pi} \frac{1}{2} |f(x) - \sum_{i=1}^{n} a_i \sigma( x + b_i) |^2 dx.  
		\end{equation}
	
The above optimization problem is usually solved by GD (or Adam). 

In [ ]:
# Helper functions
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import sys
from scipy.sparse import linalg
from pathlib import Path
pi = torch.tensor(np.pi,dtype=torch.float64)
torch.set_default_tensor_type(torch.DoubleTensor)
import matplotlib.pyplot as plt 

if torch.cuda.is_available():  
    device = "cuda" 
else:  
    device = "cpu"  
    
# 5 Point Gauss Quadrature rule, defined as global variables 
gx = torch.tensor([-0.9061798459386639927976, -0.5384693101056830910363, 0, 0.5384693101056830910363, 
     0.9061798459386639927976]).to(device)
gx = gx.view(1,-1)
gw = torch.tensor([0.2369268850561890875143, 0.4786286704993664680413, 0.5688888888888888888889, 0.4786286704993664680413,
     0.2369268850561890875143]).to(device)
gw = gw.view(-1,1) # Column vector

class model(nn.Module):
    def __init__(self, input_size, hidden_size1, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, num_classes,bias = False)

    def forward(self, x):
        u1 = self.fc2(F.relu(self.fc1(x)))
        return u1

def plot_l2_error_history(alg_name,hidden_size1,err): 
    plt.figure(dpi = 100)
    plt.title(alg_name + ': neuron number '+str(hidden_size1))
    plt.plot(err, label = 'L2 error', linewidth=1)
    plt.legend()
    plt.xlabel('epoch')
    plt.yscale('log')
    plt.ylabel('error')
    plt.show()

def plot_solution(r1,r2,model,x_test,u_true,name=None): 
    # Plot function: test results 
    u_model_cpu = model(x_test).cpu().detach()
    plt.figure(dpi = 100)
    plt.plot(x_test.cpu(),u_model_cpu,label = "nn function")
    plt.plot(x_test.cpu(),u_true.cpu(),label = "true")
    if name!=None: 
        plt.title(name)
    plt.legend()
    plt.show()
    
def compute_integration_nodes_relunn(a,b,model): 

    """
    input: 
        a,b: the interval [a,b]
        model: nn model for extracting 
        weight, bias: \sigma(weight*x + bias) is each neuron. both are 1D tensor
    return: a column vector. size: (# cols)*(# row) 
    """
    weight = model.fc1.weight.detach().squeeze()
    bias = model.fc1.bias.detach() 
    neuron_number = bias.size(0)
    node = torch.empty(neuron_number + 2).to(device)
    node[-1] = a
    node[-2] = b 
    node[0:neuron_number] = - bias / weight 
    node = node[(node <= b)] 
    node = node[(node >= a)]
    node = node.view(-1,1)
    if neuron_number < 100: # too fewer subintervalus soemtimes lead to inaccurate GQ 
        refined_node = torch.linspace(a,b,100).view(-1,1)
        node = torch.cat([node,refined_node])
    node = node.unique()
    node, indices = torch.sort(node)
    node = node.view(-1,1)
    return node

def GQ_piecewise(gw,gx,nodes,func): 
    """
    Compute piecewise Gaussian quadrature for function func on subintervals determined by nodes
    """
    n = len(nodes) - 1 
    # 1. Get the transformed integration points for each subinterval, stored in rows
    coef1 = ((nodes[1:,:] - nodes[:-1,:])/2) # n by 1  
    coef2 = ((nodes[1:,:] + nodes[:-1,:])/2) # n by 1  
    coef2_expand = coef2.expand(-1,gx.size(1)) # Expand to n by p shape, -1: keep the first dimension n , expand the 2nd dim (columns)
    integration_points = coef1@gx + coef2_expand
    integration_points = integration_points.flatten().view(-1,1) # Make it a column vector
    
    # 2. Function values on the integration points on each subinterval
    func_values = func(integration_points)
    # Modify gw, coef1 to be compatible with func_values
    gw = torch.tile(gw,(n,1)) # rows: n copies of current tensor, columns: 1 copy, no change
    # Modify coef1 to be compatible with func_values
    coef1_expand = coef1.expand(coef1.size(0),gx.size(1))    
    coef1_expand = coef1_expand.flatten().view(-1,1)
    integral_values = torch.matmul(func_values.T,(gw*coef1_expand))
    return integral_values



In [ ]:
def AdamOrGD_train(my_model, target,learning_rate,num_epochs,initialize_): 
    """Training SNN using Adam or GD

    Parameters
    ----------
    my_model : nn model class 
        Shallow neural network model 
    target : function
        the target function
    learning_rate : float 
        learning rate for neural network training 
    num_epochs : int 
        number of epochs
    initialize_ : function
        initialize the nn parameters

    Returns
    -------
    err : torch tensor
        L2 error history with respect to the epoch number 
    my_model : nn model class 
        trained nn model
    """
    
    def loss_function_inside(x): # Integrand of the loss function
        return 0.5*torch.pow(my_model(x)-target(x),2).to(device)

    start = time.time()
    hidden_size1 = my_model.fc1.bias.size(0)
    print("current hidden layer size: ",hidden_size1)
    if initialize_ != None: 
        initialize_(my_model,target)
        
    # Plot initial guess and compute initial error
    x_test = torch.linspace(-pi,pi,4096).view(4096,1).to(device) 
    u_true = target(x_test)
    plot_solution(-pi,pi,my_model,x_test,u_true)
    
    with torch.no_grad():
        node = compute_integration_nodes_relunn(-pi,pi,my_model)
        loss = GQ_piecewise(gw,gx,node,loss_function_inside)
    print("original loss", (2*loss)**0.5)
    # Define variables for data to be stored 
    err = torch.zeros(num_epochs+1, 1)
    err[0] = (2*loss)**0.5
    
    # Define optimizer 
    optimizer = optim.Adam(my_model.parameters(), lr=learning_rate)
    
    # Start the training process
    lr = learning_rate  
    for epoch in range(num_epochs):
        # Plot the numerical solution 
        if (epoch+1) % 5000 == 0: 
            plot_solution(-pi,pi,my_model,x_test,u_true)
          
        lr = learning_rate 
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr * (0.98 ** ((epoch + 1) // 1000)) # Learning rate schedule
        optimizer.zero_grad() # Clear gradient 
        
        # Define the loss function 
        node = compute_integration_nodes_relunn(-pi,pi,my_model)
        loss = GQ_piecewise(gw,gx,node,loss_function_inside)
        loss.backward() # Compute gradient 
        optimizer.step() # Gradient descent step
        with torch.no_grad(): 
            node = compute_integration_nodes_relunn(-pi,pi,my_model)
            loss = GQ_piecewise(gw,gx,node,loss_function_inside)
        err[epoch+1] = torch.pow(loss.detach()*2.0,0.5)

    # Plot L^2 error history and solution 
    plot_l2_error_history("Adam: ",hidden_size1,err)
    plot_solution(-pi,pi,my_model,x_test,u_true)
    end = time.time()
    print(str(end - start)+" s")
    return err, my_model

### Experiments

1. $f(x) = sin(x)$

2. $f(x) = sin(x) + 0.2*sin(10*x)$

In [ ]:
my_model = model(1,20,1)
def target(x):
    return torch.sin(x)
learning_rate = 0.01 
initialize_ = None 
num_epochs = 10000
err, my_model = AdamOrGD_train(my_model, target,learning_rate,num_epochs,initialize_)

In [ ]:
my_model = model(1,20,1)
def target(x):
    return torch.sin(x) + 0.2*torch.sin(10*x)
learning_rate = 0.01 
initialize_ = None 
num_epochs = 10000
err, my_model = AdamOrGD_train(my_model, target,learning_rate,num_epochs,initialize_)

In [ ]:
def target(x):
    return torch.sin(x) 
err_list = []
for neuron_num in [8,16,32,64]: 
    my_model = model(1,neuron_num,1)
    learning_rate = 0.01 
    initialize_ = None 
    num_epochs = 10000
    err, my_model = AdamOrGD_train(my_model, target,learning_rate,num_epochs,initialize_)
    err_list.append(min(err))
print(err_list)

In [ ]:
def target(x):
    return torch.sin(x) + 0.2*torch.sin(10*x)
err_list = []
for neuron_num in [8,16,32,64]: 
    my_model = model(1,neuron_num,1)
    learning_rate = 0.01 
    initialize_ = None 
    num_epochs = 10000
    err, my_model = AdamOrGD_train(my_model, target,learning_rate,num_epochs,initialize_)
    err_list.append(min(err))
print(err_list)